In [1]:
#####Note:run in python2######
# same initialization of weights because same random seed used

# Observations:
# for training deep neural network models, the accumulation of squared gradients from the beginning of
# training can result in a premature and excessive decrease in the eﬀective learning
# rate. AdaGrad performs well for some but not all deep learning models.
# When applied to a nonconvex function to train a neural network,
# the learning trajectory may pass through many diﬀerent structures and eventually
# arrive at a region that is a locally convex bowl. AdaGrad shrinks the learning rate
# according to the entire history of the squared gradient and may have made the
# learning rate too small before arriving at such a convex structure.

import math
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt 

#for shuffling data
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
#########
#MLP

# initialisation of the weights he_normal
def weights(noHiddenLayers,sizeOfLayers):

    W=[]
    b=[]

    for i in range(0,noHiddenLayers+1):
        
        W.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+sizeOfLayers[i])),
                                   (sizeOfLayers[i+1],sizeOfLayers[i])) )
        b.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+1)),
                                   (sizeOfLayers[i+1],1)) )

    W=np.array(W)
    b=np.array(b)
    
    return W,b


#mlp forward pass
#layer
def layer(w,x,b):
    out = np.matmul(w,x)+b
    return out

def apply_activationMLP(Activation_function,inp):
    
    #activation functions
    if Activation_function == 'relu':
        return np.where(inp<0,0,inp)
    elif Activation_function == "tanh":
        return np.tanh(inp)
    elif Activation_function == "sigmoid":
        return 1.0/(1+np.exp(-1.0*inp))
    elif Activation_function == "softmax":
        return (1.0/(np.sum(np.exp(inp),axis=0)))*(np.exp(inp))

#forward path
def forward_path(noHiddenLayers,X,W,b,Actfnvect):

    out=[]
    
    z=apply_activationMLP(Actfnvect[0],np.array(layer(W[0],X,b[0])))
    out.append(np.array(z))

    for i in range(1,noHiddenLayers):
        z=apply_activationMLP(Actfnvect[i],np.array(layer(W[i],out[i-1],b[i])))
        out.append(np.array(z))

    if noHiddenLayers > 0:
        z=apply_activationMLP(Actfnvect[-1],np.array(layer(W[-1],out[-1],b[-1])))
        out.append(np.array(z))

    y_pred = out[-1]

    return np.array(out),np.array(y_pred)


In [3]:
#only to import data
from keras import backend as K
import keras
from keras.datasets import mnist


# #import data
iris_data = load_iris() # load the iris dataset

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
x_train , x_test, y_train , y_test = train_test_split(x, y, test_size=0.20)

print(x_train.shape)
print(y_train.shape)

Using TensorFlow backend.


(120, 4)
(120, 3)


In [4]:
# run MLP algorithm

x_train = np.moveaxis(x_train,0,-1)
y_train = np.moveaxis(y_train,0,-1)
x_test = np.moveaxis(x_test,0,-1)
y_test = np.moveaxis(y_test,0,-1)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(4, 120)
(3, 120)
(4, 30)
(3, 30)


In [5]:
#derivative of relu
def der_relu(x):
    return np.where(x == 0,0,1)

# backpropation
def backprop(y,y_true,z,T,u,M,x):
    
    dy = (y-y_true)
    
    # layer 3
    dT  = np.matmul(dy,z.T)
    db3 = np.sum(dy,axis=1).reshape(-1,1)
    
    # layer 2
    s   = np.matmul(T.T,dy)
    s   = s*der_relu(z)
    dM  = np.matmul(s,u.T)
    db2 = np.sum(s,axis=1).reshape(-1,1)
    
    # layer 1
    sm  = np.matmul(M.T,s)
    sm  = sm*der_relu(u)
    dW  = np.matmul(sm,x.T)
    db1 = np.sum(sm,axis=1).reshape(-1,1)
    
    return dW,dM,dT,db1,db2,db3

In [6]:
###
################################
# Training parameters
num_classes = 3
epochs = 500
batch_size = 10
learning_rate = 0.01
delta=1e-7
################################

#MLP PARAMETERS
noHiddenLayers=2

#also includes the input vector dimension and output vector dimension
sizeOfLayers=[x_train.shape[0],10,10,num_classes]

sizeofOutput=num_classes

Actfnvect = ['relu','relu','softmax']

In [7]:
W,b = weights(noHiddenLayers,sizeOfLayers)

In [8]:
# (a)
# Momentum descent

# updating the weights


r = {"W[0]" : np.zeros(W[0].shape) , "W[1]" : np.zeros(W[1].shape) ,"W[2]" : np.zeros(W[2].shape),"b[0]": np.zeros(b[0].shape)
     ,"b[1]" :np.zeros(b[1].shape) , "b[2]" : np.zeros(b[2].shape)}

for i in range(epochs):
    loss=0
    for j in np.arange(0,x_train.shape[1],batch_size):       
        dW = np.zeros(W[0].shape)
        dM = np.zeros(W[1].shape)
        dT = np.zeros(W[2].shape)
        db1 = np.zeros(b[0].shape)
        db2 = np.zeros(b[1].shape)
        db3 = np.zeros(b[2].shape)
        for k in range(0,batch_size):
            # forward pass
            x = x_train[:,j+k].reshape(-1,1)
            y = y_train[:,j+k].reshape(-1,1)
            out,y_pred=forward_path(noHiddenLayers,x,W,b,Actfnvect)
            # backpropagation
            dWtemp,dMtemp,dTtemp,db1temp,db2temp,db3temp=backprop(y_pred,y,out[1],W[-1],out[0],W[1],x)
            
            dW=dW+dWtemp
            db1=db1+db1temp
            
            dM=dM+dMtemp
            db2=db2+db2temp
            
            dT=dT+dTtemp
            db3=db3+db3temp
        
            # calculate the loss
            loss = loss + (-1.0*np.sum(y*np.log(y_pred)))
            
        # Updating the weights using Adagrad Approach
        dW=dW*(1.0/batch_size)
        dM=dM*(1.0/batch_size)
        dT=dT*(1.0/batch_size)
        db1=db1*(1.0/batch_size)
        db2=db2*(1.0/batch_size)
        db3=db3*(1.0/batch_size)
        
        r["W[0]"] = r["W[0]"] + ((dW*dW))
        r["W[1]"] = r["W[1]"] + ((dM*dM))
        r["W[2]"] = r["W[2]"] + ((dT*dT))
        r["b[0]"] = r["b[0]"] + ((db1*db1))
        r["b[1]"] = r["b[1]"] + ((db2*db2))
        r["b[2]"] = r["b[2]"] + ((db3*db3))
        
        W[0] = W[0] - ( (learning_rate*dW)/(delta +np.sqrt(r["W[0]"])))
        W[1] = W[1] - ( (learning_rate*dM)/(delta +np.sqrt(r["W[1]"])))
        W[2] = W[2] - ( (learning_rate*dT)/(delta +np.sqrt(r["W[2]"])))
        b[0] = b[0] - ((learning_rate*db1)/(delta +np.sqrt(r["b[0]"])))
        b[1] = b[1] - ((learning_rate*db2)/(delta +np.sqrt(r["b[1]"])))
        b[2] = b[2] - ((learning_rate*db3)/(delta +np.sqrt(r["b[2]"])))
    
    #print the loss in each epoch
    print('Epoch:'+str(i)+'         Loss:'+str(loss))

Epoch:0         Loss:188.95941234972648
Epoch:1         Loss:115.06341514368438
Epoch:2         Loss:83.3120040708373
Epoch:3         Loss:67.55527781826775
Epoch:4         Loss:58.655780160441665
Epoch:5         Loss:53.793604069510366
Epoch:6         Loss:50.70087354903374
Epoch:7         Loss:48.47551262865901
Epoch:8         Loss:46.7294406898982
Epoch:9         Loss:45.292066368028294
Epoch:10         Loss:44.05941542133694
Epoch:11         Loss:42.96943285646006
Epoch:12         Loss:41.98166707758445
Epoch:13         Loss:41.08473998205652
Epoch:14         Loss:40.24259374454702
Epoch:15         Loss:39.37496964260255
Epoch:16         Loss:38.526671170022915
Epoch:17         Loss:37.61299552092703
Epoch:18         Loss:36.65183795360373
Epoch:19         Loss:35.7079251677687
Epoch:20         Loss:34.83941827686435
Epoch:21         Loss:34.0160820145412
Epoch:22         Loss:33.233235875094365
Epoch:23         Loss:32.45567141145997
Epoch:24         Loss:31.691722786003947
Epoch:

Epoch:203         Loss:9.907888772452372
Epoch:204         Loss:9.89096834324848
Epoch:205         Loss:9.873874101902816
Epoch:206         Loss:9.85695467284154
Epoch:207         Loss:9.84020241736283
Epoch:208         Loss:9.823613132308237
Epoch:209         Loss:9.806885324553527
Epoch:210         Loss:9.790928712511139
Epoch:211         Loss:9.774801619577005
Epoch:212         Loss:9.758833869964564
Epoch:213         Loss:9.743019017332061
Epoch:214         Loss:9.72735342186598
Epoch:215         Loss:9.711547201313472
Epoch:216         Loss:9.696580828520187
Epoch:217         Loss:9.681340429401914
Epoch:218         Loss:9.66625113913943
Epoch:219         Loss:9.651303792337428
Epoch:220         Loss:9.63649403148035
Epoch:221         Loss:9.621819099071319
Epoch:222         Loss:9.607276824228395
Epoch:223         Loss:9.592865271392668
Epoch:224         Loss:9.578582612772117
Epoch:225         Loss:9.564427080798865
Epoch:226         Loss:9.550396949961922
Epoch:227         Loss

Epoch:415         Loss:8.1057261850088
Epoch:416         Loss:8.101510901161712
Epoch:417         Loss:8.09731495008557
Epoch:418         Loss:8.09313823501995
Epoch:419         Loss:8.088979811261927
Epoch:420         Loss:8.084840582957495
Epoch:421         Loss:8.080720228283822
Epoch:422         Loss:8.076618617219165
Epoch:423         Loss:8.07253561932588
Epoch:424         Loss:8.06847110450995
Epoch:425         Loss:8.064424943425667
Epoch:426         Loss:8.060397007688074
Epoch:427         Loss:8.05638716998184
Epoch:428         Loss:8.052395304113732
Epoch:429         Loss:8.048421285034856
Epoch:430         Loss:8.044464988846396
Epoch:431         Loss:8.04052629279656
Epoch:432         Loss:8.03660507527286
Epoch:433         Loss:8.032701215791828
Epoch:434         Loss:8.028800357961476
Epoch:435         Loss:8.024932675166298
Epoch:436         Loss:8.0210817362953
Epoch:437         Loss:8.017247564193035
Epoch:438         Loss:8.013445620271874
Epoch:439         Loss:8.00

In [9]:
out,y_pred=forward_path(noHiddenLayers,x_test,W,b,Actfnvect)

In [10]:
def predict(y):
    return np.argmax(y)

yvect=[]
y_trurevect=[]

for i in range(0,x_test.shape[1]):
    yvect.append(predict(y_pred[:,i]))
    y_trurevect.append(predict(y_test[:,i]))

# find accuracy
from sklearn.metrics import accuracy_score
#predicting test accuracy
print(accuracy_score(y_trurevect, yvect))

1.0


In [11]:
# to see the output vs true values

print y_trurevect
print yvect

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
